Calcular el promedio de rating por tipo de App.

# Cargo el csv y hago ajustes necesarios.

In [1]:
! pip install pyspark
! pip install -U -q PyDrive
! apt update
! apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6c7253e60857dd29f7c157e83555a59b6d3d0703674aea3e3fb1dbacc8599081
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-upda

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.types import IntegerType
import math

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({"id" : "1-oPVltZP-8bpOqQtlRLD6yy6TNasoHLP"})
downloaded.GetContentFile("googleplaystore.csv")

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Skipeo una línea con el nombre de la aplicación respectivo porque está mal parseada.

In [6]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore.csv", header = True, inferSchema = True)
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame" and x.App != "Command & Conquer: Rivals")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Comienzo del ejercicio.

Uso take en cada celda para la revisión y sé que no es necesario.

Hay una fila con Type = "NaN" que omito al cargar el csv y no se toma en cuenta en estos cálculos.

In [7]:
rdd_rating = rdd.map(lambda x: (x.Type, (x.Rating if not math.isnan(x.Rating) else 0, 1 if not math.isnan(x.Rating) else 0)))
rdd_rating.take(5)

[('Free', (4.1, 1)),
 ('Free', (3.9, 1)),
 ('Free', (4.7, 1)),
 ('Free', (4.5, 1)),
 ('Free', (4.3, 1))]

Los ratings con NaN los tomo como 0 y al total sumo 0 también.

In [8]:
rdd_rating = rdd_rating.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
rdd_rating.collect()

[('Free', (36499.500000000095, 8719)), ('Paid', (2760.499999999999, 647))]

El promedio de rating por tipo de app (Free/Paid) es:

In [9]:
rdd_rating = rdd_rating.map(lambda x: (x[0], x[1][0] / x[1][1]))
rdd_rating.collect()

[('Free', 4.186202546163562), ('Paid', 4.266615146831529)]